### This file contains functions related to loading, cleaning, and preproessing data.


The link to where the data was dowloanded from : https://www.cryptodatadownload.com/data/poloniex/#google_vignette

In [2]:
# This code loads the data from a path

import pandas as pd

def load_data(csv_path):
    # If the first row of your CSV is just a note (e.g. “Data provided by...”),
    # then skip it with skiprows=1. If not, set skiprows=0 or remove that parameter.
    
    df = pd.read_csv(
        csv_path,
        skiprows=1,            # Adjust if your first row is already column headers
        parse_dates=['date'],  # Tells Pandas to convert the 'date' column to datetime64 objects.
        infer_datetime_format=True  #Lets Pandas guess the date format more efficiently, which can speed up parsing if your file is big.

    )
    
    #Sets the "date" column as the index of the DataFrame.
    df.set_index('date', inplace=True)
    
    #sorts the DataFrame by the date/time index, just in case the rows were out of order.
    df.sort_index(inplace=True)
    
    return df



In [ ]:
#Use of the load_data function 
path_to_csv = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\raw\2025-2015 Gemini_BTCUSD_1h.csv"
df_prices = load_data(path_to_csv)
    
print("DataFrame shape:", df_prices.shape)
print(df_prices.head())
print(df_prices.tail())

series models:

Date/Time : Usually kept as an index.

Open, High, Low, Close (OHLC) : Core price features for any trading or forecasting model.

Volume : Overall volume is often a good indicator of market interest and liquidity.

Trade Count (Optional) : Helps distinguish whether volume came from many small trades or fewer large trades.


In [4]:
# This code Keep only the columns that are most useful for ur dataframe

def keep_important_columns(df):

    # Define the columns you want to keep
    columns_to_keep = [
        "open",
        "high",
        "low",
        "close",
        "Volume BTC",  
    ]
    
    # Intersect with what actually exists in your DataFrame to avoid KeyErrors
    existing_cols = [col for col in columns_to_keep if col in df.columns]
    
    # Create a reduced DataFrame with only these columns
    df_reduced = df[existing_cols].copy()
    return df_reduced


In [ ]:
# use of keep_important_columns
df_prices = keep_important_columns(df_prices)
print(df_prices)

In [6]:
# Save the DataFrame to CSV
df_prices.to_csv('gemini_btc_data_v1')


Below is a data-quality checker , It identifies common issues in a crypto price DataFrame, such as:

Missing Values: Looks for NaN/None in columns.

Duplicate Rows: Checks whether any exact duplicates exist.

Negative or Zero Price/Volume: Flags rows where prices or volumes are invalid.

Out-of-Order Date Index: Ensures that your date index is strictly increasing (important for time-series).

Unexpected Data Types: Verifies that “open”, “high”, “low”, “close”, and “volume” columns are numeric.

In [ ]:
import pandas as pd
import numpy as np

def check_data_problems(df):
    problems_summary = {}

    # 1) Check for missing values
    missing = df.isnull().sum()
    if missing.any():
        print(">> MISSING VALUES found per column:")
        print(missing[missing > 0])
        problems_summary['missing_values'] = missing[missing > 0].to_dict()
    else:
        print("No missing values detected.")
        problems_summary['missing_values'] = {}
    
    # 2) Check for duplicate rows
    duplicate_count = df.duplicated().sum()
    if duplicate_count > 0:
        print(f">> DUPLICATE ROWS found: {duplicate_count}")
        problems_summary['duplicate_rows'] = duplicate_count
    else:
        print("No duplicate rows found.")
        problems_summary['duplicate_rows'] = 0
    
    # 3) Negative or zero price/volume checks
    price_cols = [col for col in ['open', 'high', 'low', 'close'] if col in df.columns]
    volume_cols = [col for col in ['Volume BTC', 'Volume USDT', 'Volume USD', 'volume'] if col in df.columns]

    invalid_prices = {}
    for col in price_cols:
        non_pos = (df[col] <= 0).sum()
        if non_pos > 0:
            invalid_prices[col] = non_pos
    
    invalid_volumes = {}
    for col in volume_cols:
        non_pos = (df[col] <= 0).sum()
        if non_pos > 0:
            invalid_volumes[col] = non_pos

    if invalid_prices:
        print(">> INVALID (≤0) PRICE VALUES found:")
        for c, count in invalid_prices.items():
            print(f"   Column '{c}': {count} rows")
        problems_summary['invalid_prices'] = invalid_prices
    else:
        print("No invalid (zero/negative) price values found.")
        problems_summary['invalid_prices'] = {}
    
    if invalid_volumes:
        print(">> INVALID (≤0) VOLUME VALUES found:")
        for c, count in invalid_volumes.items():
            print(f"   Column '{c}': {count} rows")
        problems_summary['invalid_volumes'] = invalid_volumes
    else:
        print("No invalid (zero/negative) volume values found.")
        problems_summary['invalid_volumes'] = {}

    # 4) Out-of-order date index check (only if index is datetime-like)
    if isinstance(df.index, pd.DatetimeIndex):
        if not df.index.is_monotonic_increasing:
            print(">> The date index is NOT strictly increasing. Some timestamps may be out of order.")
            problems_summary['date_order'] = "Not strictly increasing"
        else:
            print("Date index is in ascending order (strictly increasing).")
            problems_summary['date_order'] = "Ascending"
    else:
        print("Index is not a DatetimeIndex (skipping date-order check).")
        problems_summary['date_order'] = None

    # 5) Data-type checks for numeric columns
    numeric_checks = {}
    for col in price_cols + volume_cols:
        if col in df.columns and not pd.api.types.is_numeric_dtype(df[col]):
            numeric_checks[col] = "Non-numeric type"
    if numeric_checks:
        print(">> NON-NUMERIC COLUMNS found (expected numeric):")
        for c, msg in numeric_checks.items():
            print(f"   Column '{c}' => {msg}")
    else:
        print("All price/volume columns have numeric types.")
    problems_summary['non_numeric_columns'] = numeric_checks

    print("\n=== DATA QUALITY CHECK COMPLETE ===\n")
    return problems_summary


In [ ]:
# 3. Run the data-quality checker using your df_prices DataFrame
problems_report = check_data_problems(df_cleaned)

# 4. If you want to do something programmatic with the results:
print("Problems Summary (as dict):")
print(problems_report)

# Additionally, display the rows where invalid (≤0) PRICE VALUES are found
if 'open' in df_cleaned.columns:
    invalid_open = df_cleaned[df_cleaned['open'] <= 0]
    print("\nRows with invalid 'open' values (≤0):")
    print(invalid_open)

if 'low' in df_cleaned.columns:
    invalid_low = df_cleaned[df_cleaned['low'] <= 0]
    print("\nRows with invalid 'low' values (≤0):")
    print(invalid_low)
    

'''
No missing values detected.
>> DUPLICATE ROWS found: 480
>> INVALID (≤0) PRICE VALUES found:
   Column 'open': 1 rows
   Column 'low': 1 rows
>> INVALID (≤0) VOLUME VALUES found:
   Column 'Volume BTC': 1178 rows
Date index is in ascending order (strictly increasing).
All price/volume columns have numeric types.

=== DATA QUALITY CHECK COMPLETE ===

Problems Summary (as dict):
{'missing_values': {}, 'duplicate_rows': 480, 'invalid_prices': {'open': 1, 'low': 1}, 'invalid_volumes': {'Volume BTC': 1178}, 'date_order': 'Ascending', 'non_numeric_columns': {}}

Rows with invalid 'open' values (≤0):
                     open   high  low  close  Volume BTC
date                                                    
2015-10-08 13:00:00   0.0  245.0  0.0  245.0    0.606654

Rows with invalid 'low' values (≤0):
                     open   high  low  close  Volume BTC
date                                                    
2015-10-08 13:00:00   0.0  245.0  0.0  245.0    0.606654
'''

In [ ]:
#this code reads the dataframe from the saved csv
df = pd.read_csv(
    r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_data_v1.csv",
    index_col=0,
    parse_dates=True
)
print(df)


### Deal with Duplicate Rows

In [ ]:
# This will return every row that has a duplicate somewhere else
duplicates_all = df[df.duplicated(keep=False)]

print(f"Total rows considered duplicates (including the first occurrence): {len(duplicates_all)}")
print(duplicates_all)


In [ ]:
# Save the duplicate rows to a CSV file
#duplicates_all.to_csv("duplicates.csv", index=True)
print("Duplicate rows have been saved to duplicates.csv")

Duplicate rows have been saved to duplicates.csv


In [ ]:
# Remove duplicate rows (keeps the first occurrence)
df_dedup = df.drop_duplicates()

print("After dropping duplicates, the dataframe has", len(df_dedup), "rows.")



After dropping duplicates, the dataframe has 82520 rows.


In [26]:
# This will return every row that has a duplicate somewhere else
duplicates_all = df_dedup[df_dedup.duplicated(keep=False)]

print(f"Total rows considered duplicates (including the first occurrence): {len(duplicates_all)}")
print(duplicates_all)

Total rows considered duplicates (including the first occurrence): 0
Empty DataFrame
Columns: [open, high, low, close, Volume BTC]
Index: []


In [ ]:
#Below is an example of a separate Jupyter Notebook cell that first identifies the row(s) where the price value (for example, in the 'open' or 'low' column)
# 0 is 0 and then removes it from the DataFrame:

# Identify the row(s) where 'open' or 'low' is 0 (or less)
invalid_price_rows = df_dedup[(df_dedup['open'] <= 0) | (df_dedup['low'] <= 0)]
print("Rows with invalid price values (open or low ≤ 0):")
print(invalid_price_rows)

# Remove these rows from the DataFrame
df_cleaned = df_dedup[(df_dedup['open'] > 0) & (df_dedup['low'] > 0)]
print("\nDataFrame after removing the row(s) with invalid price values:")
print(df_cleaned)

'''
rows with invalid price values (open or low ≤ 0):
                     open   high  low  close  Volume BTC
date                                                    
2015-10-08 13:00:00   0.0  245.0  0.0  245.0    0.606654

DataFrame after removing the row(s) with invalid price values:
                         open      high       low     close  Volume BTC
date                                                                   
2015-10-08 14:00:00    245.00    245.00    244.50    245.00    4.453649
2015-10-08 15:00:00    245.00    245.00    244.92    244.92    3.016926
2015-10-08 16:00:00    244.92    244.92    244.25    244.25    3.895252
2015-10-08 17:00:00    244.25    244.99    244.02    244.99    3.920632
2015-10-08 18:00:00    244.99    244.99    244.00    244.00    3.690472
...                       ...       ...       ...       ...         ...
2025-03-27 19:00:00  87192.76  87433.40  87000.00  87063.30   40.926363
2025-03-27 20:00:00  87063.30  87335.11  86930.18  87310.36   18.377515
2025-03-27 21:00:00  87310.36  87710.97  87226.71  87551.64   18.607755
2025-03-27 22:00:00  87551.64  87603.05  87298.29  87346.45   10.438136
2025-03-27 23:00:00  87346.45  87346.45  87101.82  87223.20   12.074964

[82519 rows x 5 columns]
'''


In [28]:
# Save the DataFrame to CSV
df_cleaned.to_csv('gemini_btc_data_final_version')


In [ ]:
#Count duplicates by year and month
# Ensure index is datetime (just in case)
duplicates_all.index = pd.to_datetime(duplicates_all.index)

# Create year and month columns
duplicates_all['year'] = duplicates_all.index.year
duplicates_all['month'] = duplicates_all.index.month

# Group by year and month, then count rows
duplicate_counts = duplicates_all.groupby(['year', 'month']).size().reset_index(name='count')

# Display
print(duplicate_counts)



In [ ]:
#this code compares bettwen the 3 datasets
import pandas as pd

# -------------------------------------------------------------------
# 1) Set your CSV file paths here, named for each exchange
# -------------------------------------------------------------------
poloniex_csv = r"C:\Users\ADMIN\Downloads\Poloniex_BTCUSDT_1h.csv"
bitstamp_csv = r"C:\Users\ADMIN\Downloads\Bitstamp_BTCUSD_1h (1).csv"
gemini_csv   = r"C:\Users\ADMIN\Downloads\2025-2015 Gemini_BTCUSD_1h.csv"

# -------------------------------------------------------------------
# 2) Read each dataset into a DataFrame
# -------------------------------------------------------------------
df_poloniex = pd.read_csv(
    poloniex_csv,
    skiprows=1,
    header=0,
    sep=',',
    parse_dates=['date'],
    index_col='date'
)

df_bitstamp = pd.read_csv(
    bitstamp_csv,
    skiprows=1,
    header=0,
    sep=',',
    parse_dates=['date'],
    index_col='date'
)

df_gemini = pd.read_csv(
    gemini_csv,
    skiprows=1,
    header=0,
    sep=',',
    parse_dates=['date'],
    index_col='date'
)

# -------------------------------------------------------------------
# 3) Align the DataFrames to the common hourly timestamps
#    (only keep rows where all three have data)
# -------------------------------------------------------------------
common_index = df_poloniex.index.intersection(df_bitstamp.index).intersection(df_gemini.index)
df_poloniex = df_poloniex.loc[common_index]
df_bitstamp = df_bitstamp.loc[common_index]
df_gemini   = df_gemini.loc[common_index]

# -------------------------------------------------------------------
# 4) Combine into one comparison table
#    Renaming columns to reflect each exchange name clearly
# -------------------------------------------------------------------
comparison_df = pd.DataFrame(index=common_index)
comparison_df['Poloniex_Close'] = df_poloniex['close']
comparison_df['Bitstamp_Close'] = df_bitstamp['close']
comparison_df['Gemini_Close']   = df_gemini['close']

# -------------------------------------------------------------------
# 5) Calculate the price differences
# -------------------------------------------------------------------
comparison_df['Poloniex_minus_Bitstamp'] = comparison_df['Poloniex_Close'] - comparison_df['Bitstamp_Close']
comparison_df['Bitstamp_minus_Gemini']   = comparison_df['Bitstamp_Close'] - comparison_df['Gemini_Close']
comparison_df['Poloniex_minus_Gemini']   = comparison_df['Poloniex_Close'] - comparison_df['Gemini_Close']

# -------------------------------------------------------------------
# 6) Display the final comparison DataFrame (first 100 rows)
# -------------------------------------------------------------------
print("\n=== Comparison of Poloniex, Bitstamp, and Gemini (Hourly) ===")
print(comparison_df)

# Optionally, you can save the comparison to a CSV file:
# comparison_df.to_csv('comparison_output.csv')


In [26]:
# Save the DataFrame to CSV
#comparison_df.to_csv('comparison_csv')

In [27]:
comparison_df[['Poloniex_minus_Bitstamp', 'Bitstamp_minus_Gemini', 'Poloniex_minus_Gemini']].mean()


#Poloniex_minus_Bitstamp    -9.114675
#Bitstamp_minus_Gemini      -1.109679
#Poloniex_minus_Gemini     -10.224354



Poloniex_minus_Bitstamp    -9.114675
Bitstamp_minus_Gemini      -1.109679
Poloniex_minus_Gemini     -10.224354
dtype: float64

Poloniex_minus_Bitstamp    -9.114675
Bitstamp_minus_Gemini      -1.109679
Poloniex_minus_Gemini     -10.224354
dtype: float64

In [28]:
import numpy as np
import pandas as pd

pairs = [
    ('Poloniex_Close', 'Bitstamp_Close'),
    ('Bitstamp_Close', 'Gemini_Close'),
    ('Poloniex_Close', 'Gemini_Close')
]

for colA, colB in pairs:
    diffs = comparison_df[colA] - comparison_df[colB]
    
    mae  = np.mean(np.abs(diffs))
    rmse = np.sqrt(np.mean(diffs**2))
    corr = comparison_df[colA].corr(comparison_df[colB])
    
    print(f"--- {colA} vs {colB} ---")
    print(f"  MAE : {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  Corr: {corr:.4f}")
    print()


--- Poloniex_Close vs Bitstamp_Close ---
  MAE : 31.4501
  RMSE: 78.8900
  Corr: 1.0000

--- Bitstamp_Close vs Gemini_Close ---
  MAE : 10.5856
  RMSE: 31.9917
  Corr: 1.0000

--- Poloniex_Close vs Gemini_Close ---
  MAE : 33.4455
  RMSE: 84.3110
  Corr: 1.0000



--- Poloniex_Close vs Bitstamp_Close ---
  MAE : 31.4501
  RMSE: 78.8900
  Corr: 1.0000

--- Bitstamp_Close vs Gemini_Close ---
  MAE : 10.5856
  RMSE: 31.9917
  Corr: 1.0000

--- Poloniex_Close vs Gemini_Close ---
  MAE : 33.4455
  RMSE: 84.3110
  Corr: 1.0000


## Comparison of the most known crypto data sites

In [ ]:
import datetime
import ccxt
import pandas as pd
import time

# -----------------------------------------------------------------------------------
# 1) Choose which exchanges you want to pull data from
# -----------------------------------------------------------------------------------
exchange_ids = [
    'binance',
    'kraken',
    'bitstamp',
    'coinbase',  # coinbasepro in older ccxt versions
    'bybit',
    # Add more if needed, but note not all support the same endpoints or markets
]


# -----------------------------------------------------------------------------------
# 2) Set up a function to fetch OHLCV from an exchange using CCXT
# -----------------------------------------------------------------------------------
def fetch_ohlcv_data(exchange_id, symbol, timeframe, since_ts, limit=100):
    """
    Fetches OHLCV data from a specific exchange.

    :param exchange_id: string, CCXT exchange id (e.g. 'binance')
    :param symbol: string, market symbol on that exchange (e.g. 'BTC/USDT', 'BTC/USD')
    :param timeframe: string, resolution for OHLCV (e.g. '1h', '1d')
    :param since_ts: int, a Unix timestamp in ms from which to start fetching
    :param limit: int, maximum number of data points to fetch in one call
    :return: list of lists, each sub-list is [timestamp, open, high, low, close, volume]
    """
    try:
        exchange_class = getattr(ccxt, exchange_id)
        exchange = exchange_class()

        # Many exchanges label BTC with "BTC/USDT" or "BTC/USD".
        # Make sure the symbol is correct for the selected exchange.
        # If the exchange does not have the trading pair in this format, you'll get an error.

        # Some exchanges require authentication or other settings, but we'll try public OHLCV.
        if exchange.has['fetchOHLCV']:
            data = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since_ts, limit=limit)
            return data
        else:
            print(f"{exchange_id} does not support OHLCV fetch.")
            return []
    except Exception as e:
        print(f"Error fetching data from {exchange_id}: {e}")
        return []


# -----------------------------------------------------------------------------------
# 3) Choose your parameters
# -----------------------------------------------------------------------------------
symbol = "BTC/USDT"  # Adjust if exchange doesn’t support USDT markets (then use BTC/USD)
timeframe = "1h"  # Hourly data
since_ts = 1743195600000  # This is the timestamp you provided (in ms)
limit = 100  # How many candles you want to fetch

# Convert that timestamp to a human-readable format (optional, just for reference)
human_readable = datetime.datetime.utcfromtimestamp(since_ts / 1000.0)
print("Requesting OHLCV data starting from UTC:", human_readable)

# -----------------------------------------------------------------------------------
# 4) Fetch data from each exchange and compare
# -----------------------------------------------------------------------------------
all_dataframes = {}

for ex_id in exchange_ids:
    print(f"\n--- Fetching data from {ex_id} ---")
    ohlcv = fetch_ohlcv_data(ex_id, symbol, timeframe, since_ts, limit)

    if not ohlcv:
        print(f"No data returned from {ex_id}")
        continue

    # Convert raw OHLCV data to a Pandas DataFrame
    # CCXT returns each row in this format:
    # [timestamp, open, high, low, close, volume]
    df = pd.DataFrame(ohlcv, columns=["Timestamp", "Open", "High", "Low", "Close", "Volume"])

    # Convert timestamp to human-readable date
    df['Date'] = pd.to_datetime(df['Timestamp'], unit='ms', utc=True)
    df.set_index('Date', inplace=True)

    # Drop the 'Timestamp' column now that we have a Date index
    df.drop(columns='Timestamp', inplace=True)

    print(f"Data from {ex_id}:")
    print(df.head())  # or df.tail()

    all_dataframes[ex_id] = df

# -----------------------------------------------------------------------------------
# 5) Simple comparison: for example, compare closing prices across exchanges
# -----------------------------------------------------------------------------------
# Align all DataFrames on their Datetime index
comparison_df = pd.DataFrame(index=sorted(set().union(*(df.index for df in all_dataframes.values()))))

for ex_id, df in all_dataframes.items():
    comparison_df[ex_id + '_Close'] = df['Close']

print("\n--- Comparison of Close prices across the exchanges (first few rows) ---")
print(comparison_df.head(20))

# -----------------------------------------------------------------------------------
# 6) Further analysis could go here
# For instance, you could look for cross-exchange differences or plot them.
# -----------------------------------------------------------------------------------



Data from binance:
                               Open      High       Low     Close     Volume
Date                                                                        
2025-03-28 21:00:00+00:00  83794.41  84110.36  83697.83  84052.29  295.77685
2025-03-28 22:00:00+00:00  84052.29  84480.36  84052.29  84480.35  382.78331
2025-03-28 23:00:00+00:00  84480.35  84538.69  84200.01  84424.38  459.05926
2025-03-29 00:00:00+00:00  84424.38  84624.73  84353.49  84396.89  414.88307
2025-03-29 01:00:00+00:00  84396.89  84502.42  84216.00  84281.90  283.36385


Data from kraken:
                              Open     High      Low    Close     Volume
Date                                                                    
2025-03-28 21:00:00+00:00  83807.3  84099.9  83711.5  84025.3  12.312881
2025-03-28 22:00:00+00:00  84102.2  84449.9  84093.9  84449.3   4.074071
2025-03-28 23:00:00+00:00  84450.0  84534.7  84205.7  84396.0  11.075369
2025-03-29 00:00:00+00:00  84396.0  84587.3  84377.8  84407.5   0.748086
2025-03-29 01:00:00+00:00  84390.7  84508.4  84223.2  84230.0   2.434591


Data from bitstamp:
                              Open     High      Low    Close    Volume
Date                                                                   
2025-03-28 21:00:00+00:00  83726.0  83726.0  83726.0  83726.0  0.000931
2025-03-28 22:00:00+00:00  84223.0  84341.0  84223.0  84341.0  0.007428
2025-03-28 23:00:00+00:00  84513.0  84513.0  84223.0  84226.0  0.022320
2025-03-29 00:00:00+00:00  84462.0  84462.0  84461.0  84461.0  0.002750
2025-03-29 01:00:00+00:00  84430.0  84430.0  84254.0  84399.0  0.005507


Data from coinbase:
                               Open      High       Low     Close    Volume
Date                                                                       
2025-03-28 21:00:00+00:00  83805.25  84107.00  83697.39  84087.86  2.908945
2025-03-28 22:00:00+00:00  84124.54  84476.20  84122.81  84476.20  3.051222
2025-03-28 23:00:00+00:00  84466.08  84531.77  84203.13  84419.96  2.055998
2025-03-29 00:00:00+00:00  84376.51  84606.01  84356.25  84376.45  3.095949
2025-03-29 01:00:00+00:00  84379.74  84504.23  84222.85  84336.78  1.127938


Data from bybit:
                              Open      High      Low    Close      Volume
Date                                                                      
2025-03-28 21:00:00+00:00  83803.7  84118.00  83700.1  84058.2  138.322948
2025-03-28 22:00:00+00:00  84058.2  84484.90  84058.2  84484.9  168.946515
2025-03-28 23:00:00+00:00  84484.9  84535.69  84200.1  84421.3  153.906801
2025-03-29 00:00:00+00:00  84421.3  84613.40  84344.0  84389.8  148.371134
2025-03-29 01:00:00+00:00  84389.8  84495.90  84200.1  84269.2  202.086521